## Веб-скрапинг


In [69]:
import requests
from bs4 import BeautifulSoup
import json

data = {}
url1 = 'https://lobachevskii-dml.ru/journal/kuipm'
response = requests.get(url1)
soup = BeautifulSoup(response.text, features='html.parser')
issues = soup.find_all('td')
issues_links = []
pdf_links = []
i = 0
start = -1
for issue in issues:
  while True:
    start = str(issue).find('href="', start + 1)
    if start == -1:
      break
    end = start + 6 + 39 if str(issue)[start + 6 + 39] == '"' else start + 6 + 38
    issues_links.append(str(issue)[start + 6:end])
for issue_link in issues_links:
  url2 = 'https:' + issue_link
  response = requests.get(url2)
  soup = BeautifulSoup(response.text, features='html.parser')
  articles = soup.find_all('div', class_='articleItem')
  links = []
  for article in articles:
    start = str(article).find('href="') + 6
    end = start + 39 if str(issue)[start + 35] == '"' else start + 34
    links.append(str(article)[start:end])
  print(links)
  for link in links:
    id = link[-4:]
    url = 'https:' + link
    response = requests.get(url)
    soup = BeautifulSoup(response.text, features='html.parser')
    title = soup.find_all('h3', id='title')
    authors = soup.find_all('div', id='authors')
    authors = authors[0].text.replace(u'\xa0', u' ').replace('\n', '')
    title_article = str(title[0].text.replace(u'\xa0', u' ') + ' ' + authors)
    data[str(id)] = []
    data[str(id)].append({
        'title': title_article,
        'link': link
    })
    soup = BeautifulSoup(response.text, features='html.parser')
    span = soup.find_all('span', class_='glyphicon-paperclip')
    start = str(span[0].parent).find('href="') + 6
    end = str(span[0].parent).find('>') - 13
    pdf_links.append(str(span[0].parent).replace('&amp;', '&')[start:end])
with open('data.txt', 'w') as outfile:
    json.dump(data, outfile, indent=4, ensure_ascii=False)

['//lobachevskii-dml.ru/article/9712', '//lobachevskii-dml.ru/article/9714', '//lobachevskii-dml.ru/article/9715', '//lobachevskii-dml.ru/article/9716', '//lobachevskii-dml.ru/article/9717', '//lobachevskii-dml.ru/article/9718', '//lobachevskii-dml.ru/article/9719', '//lobachevskii-dml.ru/article/9720', '//lobachevskii-dml.ru/article/9721', '//lobachevskii-dml.ru/article/9722', '//lobachevskii-dml.ru/article/9723', '//lobachevskii-dml.ru/article/9725', '//lobachevskii-dml.ru/article/9726', '//lobachevskii-dml.ru/article/9727', '//lobachevskii-dml.ru/article/9728', '//lobachevskii-dml.ru/article/9729']
['//lobachevskii-dml.ru/article/9700', '//lobachevskii-dml.ru/article/9701', '//lobachevskii-dml.ru/article/9703', '//lobachevskii-dml.ru/article/9704', '//lobachevskii-dml.ru/article/9705', '//lobachevskii-dml.ru/article/9706', '//lobachevskii-dml.ru/article/9707', '//lobachevskii-dml.ru/article/9708', '//lobachevskii-dml.ru/article/9709', '//lobachevskii-dml.ru/article/9710', '//lobache

In [72]:
print(len(pdf_links))

294


## Необходимые библиотеки + список стоп-слов


In [73]:
!pip install PyMuPDF

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [74]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [75]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

from string import punctuation
punctuation
my_words = ['выпуск','который', 'некоторый', 'далее', 'общероссийский', 'математический', 'портал', 'использование', 'подразумевать', 'прочитать', 'согласный', 'соглашение',
            'параметр', 'загрузка', 'апрель', 'волёд', 'рисс', 'ностный', 'нетрудно', 'заметить', 'итак', 'следующий', 'опра', 'гато', 'нала', 'абель', 'поэтому', 'ревуяьтатшть',
            'явшть', ' етчашщий', 'еледуащш', 'услсвид', 'деавтеттотьвда', 'тагир', 'арслан', 'ренциальный', 'глушенков', 'смирнов', 'хороший', 'усоа', 'семёнов', 'айзенберг',
            'дубровин', 'анастасьев', 'отсюда', 'подп', 'никитин', 'насыров', 'прикл', 'ским', 'нечный', 'пользов', 'тельским', 'уоловжь', 'щюцедура', 'ттпг', 'чилапа', 'гумент',
            'тивный', 'лямин', 'заботин','отыс', 'каний', 'филипп', 'жения', 'ность', 'функ', 'обсуж', 'бирать', 'дящий', 'морозов', 'звягин', 'ляться', 'щюизводный', 'липшица',
            'каадый', 'разр' ,'либо', 'удовлет', 'ворять', 'бадрий', 'ственность', 'стический', 'аицева', 'заключение', 'лизащш', 'вырожа', 'зарайский', 'булат', 'григорьев',
            'венство', 'ничение', 'федяев', 'саамов', 'мазо', 'меаду', 'алнй', 'адгам', 'табла', 'исмагил', 'ниитэхий', 'рица', 'щённый', 'возм', 'производ', 'оцногп', 'ксторвас',
            'етчашщий', 'таотдаспш', 'лшйяй', 'дотмн', 'цессать', 'ставдшанюго', 'дения', 'шение', 'очень', 'ратор', 'ческий', 'изобра', ' лишь', 'михайлов', 'ление', 'альный',
            'стро', 'зации', 'органя', 'значит', 'енной', 'ственный', 'пространа']
noise = stopwords.words('russian') + list(punctuation) + list(my_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [76]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Подготовка текстового набора для векторизации

In [77]:
import fitz # install using: pip install PyMuPDF

def get_pdfs(filename):
  with fitz.open(filename) as doc:
    text = ""
    for page in doc:
        text += page.get_text().lower()
  return text

import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def pre_process(text):

    text=text.lower()

    text = word_tokenize(text)

    index = 0
    for i in range(len(text)):
      if(index!=-1):
        if (text[i] == 'л'):
          if (text[i+1] == 'и'):
            if (text[i+2] == 'т'):
              index=-1
              del text[i:len(text)]


    text = ' '.join(text)
    text=re.sub("(\\d|\\W|[a-zA-Z]|[_])+"," ",text)
    text = word_tokenize(text)

    for i in range(len(text)):
      text[i] = morph.parse(text[i])[0].normal_form

    text.pop(0)
    text = ' '.join(word for word in text if (not word in noise) and len(word)>3)

    return text


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import io
import requests
from PyPDF2 import PdfReader, PdfWriter
headers = {'User-Agent': 'Mozilla/5.0 (X11; Windows; Windows x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36'}

texts = []
for item in pdf_links:
  url = str(item)
  response = requests.get(url=url, headers=headers, timeout=120)
  on_fly_mem_obj = io.BytesIO(response.content)
  pdf_file = PdfReader(on_fly_mem_obj)
  writer = PdfWriter()
  for pageNum in range(len(pdf_file.pages)):
    currentPage = pdf_file.pages[pageNum]
    #currentPage.mergePage(watermark.getPage(0))
    writer.add_page(currentPage)
  outputStream = open("output.pdf","wb")
  writer.write(outputStream)
  outputStream.close()

  pdf_text = get_pdfs("output.pdf")
  pre_process_text = pre_process(pdf_text)
  texts.append(pre_process_text)
  !rm output.pdf

## Построение векторной модели


In [79]:
def findNumOfWords(texts):
  numOfWords = dict.fromkeys(uniqueWords, 0)
  for word in texts:
    numOfWords[word] += 1
  return numOfWords

In [80]:
def computeTF(text,numOfWord):
  tfDict = {}
  bagOfWordsCount = len(text)
  for word, count in numOfWord.items():
      tfDict[word] = count / float(bagOfWordsCount)
  return tfDict


In [81]:
def computeIDF(texts):
    import math

    max_val = 0

    idfDict = dict.fromkeys(texts[0].keys(), 0)
    for document in texts:
        for word, val in document.items():
            if val > 0:
                idfDict[word] += 1

    for word, val in idfDict.items():
      if val > max_val:
        max_val = val

    for word, val in idfDict.items():
        idfDict[word] = math.log(max_val / (1 + float(val)))
    return idfDict

In [82]:
def computeTFIDF(tfBagOfWords, idfs):
    tfidf = {}
    for word, val in tfBagOfWords.items():
        tfidf[word] = val * idfs[word]
    return tfidf

In [83]:
import nltk

for i in range(len(texts)):
  text = []
  nltk_tokens = nltk.word_tokenize(texts[i])
  for item 	in list(nltk.bigrams(nltk_tokens)):
    token = ' '.join(item)
    text.append(token)
  texts[i] = text

uniqueWords = set(texts[0]).union(set(texts[1]))
for i in range(2, len(texts)):
  uniqueWords = (set(texts[i]).union(uniqueWords))
print(len(uniqueWords))

numOfWords = []
for i in range(len(texts)):
  numOfWords.append(findNumOfWords(texts[i]))

tf = []
for i in range(len(texts)):
  tf.append(computeTF(texts[i],numOfWords[i]))

idfs = computeIDF(numOfWords)

tfidf = []
for i in range(len(tf)):
  tfidf.append(computeTFIDF(tf[i], idfs))

for i in range(len(tfidf)):
  tfidf[i] = sorted(tfidf[i].items(), key=lambda item: item[1], reverse=True)


99516


In [84]:
vector = []
for i in range(len(tfidf)):
  top_10_keyword = tfidf[i][0:10]
  keywords = {}
  keyword = []
  for k,v in top_10_keyword:
    keywords[k] = v
  for k in keywords:
    keyword.append(keywords[k])
  vector.append(keyword)

## Добавление векторов для каждого документа в json-файл

In [ ]:
id = []
with open('data.txt') as json_file:
  data = json.load(json_file)
  for item in data:
    id.append(item)


In [87]:
def add_to_json(id, vector_article):
    json_data = {
        "vector": vector_article
    }
    data = json.load(open("data.txt"))
    data[str(id)].append(json_data)
    with open("data.txt", "w") as file:
        json.dump(data, file, indent=4, ensure_ascii=False)

i = 0
for item in id:
  add_to_json(item, vector[i])
  i+=1

## Поиск близких документов

In [88]:
import math

def cosine_sim(vec1, vec2):

    dot_prod = 0
    for i, v in enumerate(vec1):
        dot_prod += v * vec2[i]

    mag_1 = math.sqrt(sum([x**2 for x in vec1]))
    mag_2 = math.sqrt(sum([x**2 for x in vec2]))

    return dot_prod / (mag_1 * mag_2)

In [ ]:
from scipy.spatial import distance

articles = []
with open('data.txt') as json_file:
  data = json.load(json_file)
  for item in id:
    p = data[str(item)][1]
    query_features = p['vector']

    distances = {}
    sorted_dict = {}

    for item in data:
      p = data[str(item)][1]
      distances[str(item)] = cosine_sim(query_features, p['vector'])
    sorted_tuples = sorted(distances.items(), key=lambda item: item[1], reverse=False)
    sorted_dict = {k: v for k, v in sorted_tuples}

    keys = []
    for key in sorted_dict:
      keys.append(key)

    query_doc = data[str(item)][0]['title']
    string = []
    for i in range(1, 6):
      info = []
      key = str(keys[i])
      return_doc = data[str(key)][0]['title']
      url = 'https:' + str(data[str(key)][0]['link'])
      info.append(return_doc)
      info.append(url)
      string.append(info)
    articles.append(string)

In [92]:
def add_to_json(id, articles):
    json_data = {
        "similar_articles": articles
    }
    data = json.load(open("data.txt"))
    data[str(id)].append(json_data)
    with open("data.txt", "w") as file:
        json.dump(data, file, indent=4, ensure_ascii=False)

i = 0
for item in id:
  add_to_json(item, articles[i])
  i+=1